In [1]:
import sys 
sys.path.append('../scripts/')
from kf import *   #誤差楕円を描くのに利用

In [2]:
def draw(xs, Rs):  ###graphslam3draw
    ##世界の描画##
    fig = plt.figure(figsize=(4,4))
    ax = fig.add_subplot(111) 
    ax.set_aspect('equal')
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5) 
    ax.set_xlabel("X",fontsize=10) 
    ax.set_ylabel("Y",fontsize=10)  
    
    ##軌跡の描画##
    poses = [xs[s] for s in range(len(hat_xs))]
    ax.scatter([e[0] for e in poses], [e[1] for e in poses], s=5, marker=".", color="black")
    ax.plot([e[0] for e in poses], [e[1] for e in poses], linewidth=0.5, color="black")
    
    ##R_tの描画##  #追加
    for t in range(1,t_end+1):
        ax.add_patch( sigma_ellipse(xs[t], Rs[t], 3) )
    
    ##描画実行##
    plt.show()

In [3]:
###データの読み込み###
delta = 0.0
xs = {}     #軌跡のデータ（ステップ数をキーにして姿勢を保存）
us = {}    #制御入力のデータ（ステップ数をキーにして保存）

with open("log.txt") as f:
    for line in f.readlines():
        tmp = line.rstrip().split()
        
        step = float(tmp[1])
        if tmp[0] == "delta":
            delta = float(tmp[1])
        elif tmp[0] == "x": #姿勢のレコードの場合
            xs[step] = np.array([float(tmp[2]), float(tmp[3]), float(tmp[4])]).T
        elif tmp[0] == "u": #制御入力の場合
            us[step] = np.array([float(tmp[2]), float(tmp[3])]).T
            
import copy
hat_xs = copy.copy(xs)

In [4]:
##空の精度行列と一次の項のベクトルの準備## 
t_end = len(xs) - 1 #step数（数式中のT）
#t_end = 1

##精度行列の次元を求めて精度行列と一次の項のベクトルの初期化##
dim = (t_end + 1)*3# + len(observed_landmarks)*2
Omega = np.zeros((dim, dim))
xi = np.zeros((dim, 1))

##確認##
print(t_end)                           #終了時刻
print(Omega)                        #精度行列
print(xi)                                  #一次の項のベクトル

41
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [5]:
##Ω_0を作る## ###graphslam3omzero
alpha = 0.001
Sigma_0 = np.diag([0.001**2, 0.001**2, 0.001**2])

Om_0 = np.linalg.inv(Sigma_0) #3x3の精度行列としてΩ0を作る
xi_0 = Om_0.dot(xs[0])

print("Ω0 = ", Om_0)
print("ξ0 =", xi_0)

Ω0 =  [[1000000.       0.       0.]
 [      0. 1000000.       0.]
 [      0.       0. 1000000.]]
ξ0 = [       0. -3000000.        0.]


In [6]:
##Ωxx,ξxxを作る##   ###graphslam3motioninfo
motion_noise_stds = {"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}

def R(t): #R_tを返す
    theta = xs[t-1][2]
    nu, om = us[t]
#    if abs(om) < 1e-5: om = 1e-5 #ゼロにすると式が変わるので避ける
#    if abs(nu) < 1e-5:         nu = 1e-5
        
    st, ct = math.sin(theta), math.cos(theta)
    stw, ctw = math.sin(theta + om*delta), math.cos(theta + om*delta)
    A = np.array([[(stw - st)/om,    -nu/(om**2)*(stw - st) + nu/om*delta*ctw],
                                 [(-ctw + ct)/om, -nu/(om**2)*(-ctw + ct) + nu/om*delta*stw],
                                 [0,                                delta]] )
    M = np.diag(
        [motion_noise_stds["nn"]**2*abs(nu)/delta +motion_noise_stds["no"]**2*abs(om)/delta,
             motion_noise_stds["on"]**2*abs(nu)/delta + motion_noise_stds["oo"]**2*abs(om)/delta]
    )
    return A.dot(M).dot(A.T) + np.diag([0.01**2, 0.01**2, 0.01**2]) #xyθ方向に誤差を混ぜておく
    #return np.diag([0.1**2, 0.1**2, 0.1**2]) 

def F(t): #F_x_{t-1}を返す（添字注意）
    F = np.array(np.eye(3))
    theta = xs[t-1][2]
    nu, om = us[t-1]
#    if abs(om) < 1e-5: om = 1e-5 #ゼロにすると式が変わるので避ける
#    if abs(nu) < 1e-5:         nu = 1e-5
    
    F[0, 2] = nu / om * (math.cos(theta + om * delta) - math.cos(theta))
    F[1, 2] = nu / om * (math.sin(theta + om * delta) - math.sin(theta))
        
    return F

##必要な行列を先に計算##
Rs, Fs, Rinvs = {}, {}, {}
for t in range(1, t_end+1):
    Rs[t] = R(t)
    Rinvs[t] = np.linalg.inv(Rs[t])
    Fs[t-1] = F(t)
    
##各x_{t-1}, x_tペアの精度行列と一次の項のベクトルを作る##
Omxx_ul, Omxx_ur, Omxx_bl, Omxx_br = {}, {}, {}, {} #x_{t-1}とx_tに対する精度行列を保管する辞書
xixx_u, xixx_b = {}, {}

import copy

for t in range(1, t_end+1):
    Omxx_ul[(t-1, t)] = Fs[t-1].T.dot(Rinvs[t]).dot(Fs[t-1])
    Omxx_ur[(t-1, t)] = -Fs[t-1].T.dot(Rinvs[t])
    Omxx_bl[(t-1, t)] = -Rinvs[t].dot(Fs[t-1].T)
    Omxx_br[(t-1, t)] = copy.copy(Rinvs[t])
    
    xixx_b[(t-1, t)] = np.array([0,0,0])#Rinvs[t].dot(hat_xs[t]- Fs[t-1].dot(xs[t-1])) #下の3行から計算
    xixx_u[(t-1, t)] = np.array([0,0,0])#- Fs[t-1].T.dot(xixx_b[(t-1, t)])

KeyError: 4

In [ ]:
Omega = np.zeros((dim, dim))
xi = np.zeros((dim, 1))

Omega[0:3, 0:3] += Om_0
xi[0:3, 0] += xi_0

for t in range(1, t_end+1):
    Omega[(t-1)*3:t*3, (t-1)*3:t*3] +=  Omxx_ul[(t-1, t)]
    Omega[(t-1)*3:t*3, t*3:(t+1)*3] +=  Omxx_ur[(t-1, t)]
    Omega[t*3:(t+1)*3, (t-1)*3:t*3] +=  Omxx_bl[(t-1, t)]
    Omega[t*3:(t+1)*3, t*3:(t+1)*3] +=  Omxx_br[(t-1, t)]
    
    xi[(t-1)*3:t*3, 0] += xixx_u[(t-1, t)]
    xi[t*3:(t+1)*3, 0] += xixx_b[(t-1, t)]
    
#xx = np.zeros((dim, 1))
#for t in range(t_end+1):
#    xx[t*3:t*3+3, 0] += xs[t]
#xi = Omega.dot(xx) 

In [ ]:
A = Omega[0:(t_end+1)*3 , 0:(t_end+1)*3 ]
b = xi[0:(t_end+1)*3, 0]

#A = Omega[0:6 , 0:6 ]
#b = xi[0:6, 0]

print(hat_xs[1])

c = np.linalg.inv(A).dot(b)
for t in range(1,t_end+1):
    hat_xs[t] = c[t*3:t*3+3]
    
print(hat_xs[1])
    
draw(hat_xs, Rs)